In [1]:
%pip install pandas
%pip install numpy
%pip install faker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------------------- ------------ 1.3/1.9 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 10.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import random
from faker import Faker

fake = Faker()

# Số lượng tài khoản
num_accounts = 5000

# Tạo danh sách các roles
roles = ['admin', 'user', 'moderator']

# Tạo dữ liệu ảo
data = []
for _ in range(num_accounts):
    account = {
        'ID': fake.uuid4(),
        'FullName': fake.name(),
        'DOB': fake.date_of_birth(minimum_age=18, maximum_age=90).strftime('%Y-%m-%d'),
        'CreatedAt': fake.date_this_decade().strftime('%Y-%m-%d'),
        'UserName': fake.email(),
        'Roles': random.choice(roles),
    }
    data.append(account)

# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(data)

# Lưu dữ liệu vào file CSV
df.to_csv('account_data.csv', index=False, encoding='utf-8-sig')

print("Dữ liệu đã được tạo và lưu vào file account_data.csv")

Dữ liệu đã được tạo và lưu vào file account_data.csv


In [2]:
import pyodbc

# Kết nối đến cơ sở dữ liệu MSSQL
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost,1433;"
    "DATABASE=master;"
    "UID=sa;"
    "PWD=Khanh076942542"
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


In [3]:

# Tạo bảng accounts nếu chưa tồn tại
cursor.execute('''
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='accounts' AND xtype='U')
CREATE TABLE accounts (
    ID NVARCHAR(50) PRIMARY KEY,
    FullName NVARCHAR(100),
    DOB DATE,
    CreatedAt DATE,
    UserName NVARCHAR(100),
    Roles NVARCHAR(50)
)
''')

# Chèn dữ liệu từ DataFrame vào bảng accounts
for index, row in df.iterrows():
    cursor.execute('''
    INSERT INTO accounts (ID, FullName, DOB, CreatedAt, UserName, Roles)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (row['ID'], row['FullName'], row['DOB'], row['CreatedAt'], row['UserName'], row['Roles']))

# Lưu các thay đổi và đóng kết nối
conn.commit()
conn.close()

print("Dữ liệu đã được chèn vào bảng accounts trong file accounts.db")

Dữ liệu đã được chèn vào bảng accounts trong file accounts.db
